In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import os
import pickle
import datetime
import csv

np.random.seed(42)
tf.random.set_seed(42)

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
    df = df.set_index('date')
    return df

def create_forecast_sequences(data, seq_length, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+forecast_horizon])
    return np.array(X), np.array(y)

def calculate_metrics(y_true, y_pred, y_lower, y_upper):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    y_lower = y_lower.flatten()
    y_upper = y_upper.flatten()
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    within_interval = np.logical_and(y_true >= y_lower, y_true <= y_upper)
    picp = np.mean(within_interval)
    mpiw = np.mean(y_upper - y_lower)
    
    return rmse, picp, mpiw

def create_train_model(X_train, y_train, forecast_horizon, n_features, model_dir, bootstrap_id):
    os.makedirs(model_dir, exist_ok=True)
    
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], n_features)),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(forecast_horizon * n_features)
    ])

    model.compile(optimizer='adam', loss='mse')

    checkpoint_path = os.path.join(model_dir, f"model_{bootstrap_id}.keras")
    
    checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        save_weights_only=False,
        verbose=0
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[checkpoint, early_stopping],
        verbose=0
    )

    return model, history

def generate_bootstrap_forecasts(X_train, y_train, X_val, forecast_horizon, n_components, 
                                model_dir, num_bootstraps=5):
    os.makedirs(model_dir, exist_ok=True)
    
    forecasts = []
    models = []

    for i in range(num_bootstraps):
        indices = np.random.choice(len(X_train), len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        model, _ = create_train_model(X_bootstrap, y_bootstrap, forecast_horizon, n_components, model_dir, i)
        models.append(model)

        forecast = model.predict(X_val, verbose=0)
        forecasts.append(forecast)

    forecasts = np.array(forecasts)
    mean_forecast = np.mean(forecasts, axis=0)
    lower_bound = np.quantile(forecasts, 0.05, axis=0)
    upper_bound = np.quantile(forecasts, 0.95, axis=0)

    return mean_forecast, lower_bound, upper_bound, models, forecasts

def generate_future_dates(start_date, months):
    return [start_date + pd.DateOffset(months=i) for i in range(months)]

def plot_forecast(df, val_dates, mean_forecast_inv, lower_bound_inv, upper_bound_inv, 
                 future_dates, future_forecast, future_lower, future_upper,
                 cycle_24_start, cycle_25_start, forecast_horizon, target_var,
                 output_dir, cycle_count, seq_length, avg_rmse, avg_picp, avg_mpiw):
    start_date = pd.to_datetime('1996-01-01')
    plt.figure(figsize=(15, 8))

    historical_mask = df.index >= start_date
    plt.plot(df.index[historical_mask], df[target_var][historical_mask], 'b-', label=f'Historical {target_var} Data')

    for i in range(len(mean_forecast_inv)):
        pred_dates = val_dates[i:i+forecast_horizon]
        if len(pred_dates) == len(mean_forecast_inv[i]):
            plt.plot(pred_dates, mean_forecast_inv[i], 'r-', alpha=0.5, label=f'Validation {target_var} Predictions' if i == 0 else "")
            plt.fill_between(pred_dates, lower_bound_inv[i], upper_bound_inv[i], color='r', alpha=0.2, label='90% Confidence Interval' if i == 0 else "")

    plt.plot(future_dates[:len(future_forecast)], future_forecast, 'g-', label=f'Future {target_var} Forecast')
    plt.fill_between(future_dates[:len(future_forecast)], future_lower, future_upper, color='g', alpha=0.2, label='90% Future Confidence Interval')

    plt.axvline(x=cycle_24_start, color='k', linestyle='--', label='Cycle 24 Start')
    plt.axvline(x=cycle_25_start, color='k', linestyle='--', label='Cycle 25 Start')
    plt.title(f'{target_var} Forecast - Cycles:{cycle_count} Seq:{seq_length}mo Hor:{forecast_horizon}mo\nRMSE:{avg_rmse:.4f} PICP:{avg_picp:.4f} MPIW:{avg_mpiw:.4f}')
    plt.xlabel('Date')
    plt.ylabel(f'{target_var}')
    plt.legend()
    plt.grid(True)
    filename = f'cycles{cycle_count}_seq{seq_length}_hor{forecast_horizon}.png'
    plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

def main():
    ssn_df = load_data('/Users/suryatejachalla/Projects/Solar-Data-Analysis/Data/processed/sunspot.csv')
    tsi_df = load_data('/Users/suryatejachalla/Projects/Solar-Data-Analysis/Data/processed/tsi.csv')
    
    df = pd.merge(ssn_df, tsi_df, left_index=True, right_index=True, suffixes=('', '_tsi'))
    df = df.rename(columns={'tsi_tsi': 'tsi'})
    
    cycle_dates = {
        19: pd.to_datetime('1954-04-01'),
        20: pd.to_datetime('1964-10-01'),
        21: pd.to_datetime('1976-03-01'),
        22: pd.to_datetime('1986-09-01'),
        23: pd.to_datetime('1996-08-01'),
        24: pd.to_datetime('2008-12-01'),
        25: pd.to_datetime('2019-12-01')
    }
    
    ssn_output_dir = "ssn_forecasts"
    tsi_output_dir = "tsi_forecasts"
    os.makedirs(ssn_output_dir, exist_ok=True)
    os.makedirs(tsi_output_dir, exist_ok=True)
    
    features = df[['ssn', 'tsi']].values
    
    n_components = 2
    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(features)
    
    pca_scaler = MinMaxScaler(feature_range=(0, 1))
    pca_scaled = pca_scaler.fit_transform(pca_features)
    
    cycle_options = [19, 20, 21, 22, 23]
    seq_length_options = [60, 72, 84, 96, 108, 120, 132]
    forecast_horizon_options = [1, 3, 6]
    
    validation_start = pd.to_datetime('2021-01-01')
    val_data = pca_scaled[(df.index >= validation_start)]
    val_dates = df.index[(df.index >= validation_start)]
    
    results = []
    total_combinations = len(cycle_options) * len(seq_length_options) * len(forecast_horizon_options)
    combination_count = 0
    
    with open('forecast_results.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['combination', 'cycle_start', 'seq_length_months', 'forecast_horizon', 
                         'ssn_rmse', 'ssn_picp', 'ssn_mpiw', 
                         'tsi_rmse', 'tsi_picp', 'tsi_mpiw'])
    
    for cycle_start in cycle_options:
        for seq_length in seq_length_options:
            for forecast_horizon in forecast_horizon_options:
                combination_count += 1
                print(f"Running combination {combination_count}/{total_combinations}: Cycle {cycle_start}, Seq Length {seq_length}, Horizon {forecast_horizon}")
                
                try:
                    train_start_date = cycle_dates[cycle_start]
                    train_end_date = cycle_dates[25]
                    
                    train_data = pca_scaled[(df.index >= train_start_date) & (df.index < train_end_date)]
                    
                    if len(train_data) <= seq_length + forecast_horizon:
                        print(f"Skipping - insufficient data for this combination")
                        continue
                    
                    model_dir = f"models_cycle{cycle_start}_seq{seq_length}_hor{forecast_horizon}"
                    
                    X_train, y_train = create_forecast_sequences(train_data, seq_length, forecast_horizon)
                    
                    y_train_flattened = y_train.reshape(y_train.shape[0], -1)
                    
                    X_val = []
                    y_val_pc = []
                    
                    for i in range(len(val_dates) - forecast_horizon + 1):
                        start_idx = df.index.get_loc(val_dates[i]) - seq_length
                        end_idx = df.index.get_loc(val_dates[i])
                        forecast_end_idx = df.index.get_loc(val_dates[i]) + forecast_horizon
                        
                        if start_idx >= 0:
                            X_val.append(pca_scaled[start_idx:end_idx])
                            y_val_pc.append(pca_scaled[end_idx:forecast_end_idx])
                    
                    X_val = np.array(X_val)
                    y_val_pc = np.array(y_val_pc)
                    
                    if len(X_val) == 0:
                        print(f"Skipping - cannot create validation sequences for this combination")
                        continue
                    
                    num_bootstraps = 5
                    mean_forecast, lower_bound, upper_bound, models, forecasts = generate_bootstrap_forecasts(
                        X_train, y_train_flattened, X_val, forecast_horizon, n_components, 
                        model_dir, num_bootstraps=num_bootstraps
                    )
                    
                    mean_forecast_reshaped = mean_forecast.reshape(-1, forecast_horizon, n_components)
                    lower_bound_reshaped = lower_bound.reshape(-1, forecast_horizon, n_components)
                    upper_bound_reshaped = upper_bound.reshape(-1, forecast_horizon, n_components)
                    
                    mean_forecast_scaled = np.array([pca_scaler.inverse_transform(f) for f in mean_forecast_reshaped])
                    lower_bound_scaled = np.array([pca_scaler.inverse_transform(f) for f in lower_bound_reshaped])
                    upper_bound_scaled = np.array([pca_scaler.inverse_transform(f) for f in upper_bound_reshaped])
                    
                    mean_forecast_original = np.array([pca.inverse_transform(f) for f in mean_forecast_scaled])
                    lower_bound_original = np.array([pca.inverse_transform(f) for f in lower_bound_scaled])
                    upper_bound_original = np.array([pca.inverse_transform(f) for f in upper_bound_scaled])
                    
                    ssn_mean_forecast_inv = np.array([f[:, 0] for f in mean_forecast_original])
                    ssn_lower_bound_inv = np.array([f[:, 0] for f in lower_bound_original])
                    ssn_upper_bound_inv = np.array([f[:, 0] for f in upper_bound_original])
                    
                    tsi_mean_forecast_inv = np.array([f[:, 1] for f in mean_forecast_original])
                    tsi_lower_bound_inv = np.array([f[:, 1] for f in lower_bound_original])
                    tsi_upper_bound_inv = np.array([f[:, 1] for f in lower_bound_original])
                    
                    ssn_actual_values = []
                    tsi_actual_values = []
                    
                    for i in range(len(y_val_pc)):
                        pc_inv = pca_scaler.inverse_transform(y_val_pc[i])
                        original_inv = pca.inverse_transform(pc_inv)
                        
                        ssn_inv = original_inv[:, 0]
                        tsi_inv = original_inv[:, 1]
                        
                        ssn_actual_values.append(ssn_inv)
                        tsi_actual_values.append(tsi_inv)
                    
                    ssn_actual_values = np.array(ssn_actual_values)
                    tsi_actual_values = np.array(tsi_actual_values)
                    
                    ssn_avg_rmse, ssn_avg_picp, ssn_avg_mpiw = 0, 0, 0
                    tsi_avg_rmse, tsi_avg_picp, tsi_avg_mpiw = 0, 0, 0
                    
                    for i in range(len(ssn_actual_values)):
                        if i < len(ssn_mean_forecast_inv):
                            ssn_rmse, ssn_picp, ssn_mpiw = calculate_metrics(
                                ssn_actual_values[i],
                                ssn_mean_forecast_inv[i],
                                ssn_lower_bound_inv[i],
                                ssn_upper_bound_inv[i]
                            )
                            
                            tsi_rmse, tsi_picp, tsi_mpiw = calculate_metrics(
                                tsi_actual_values[i],
                                tsi_mean_forecast_inv[i],
                                tsi_lower_bound_inv[i],
                                tsi_upper_bound_inv[i]
                            )
                            
                            ssn_avg_rmse += ssn_rmse
                            ssn_avg_picp += ssn_picp
                            ssn_avg_mpiw += ssn_mpiw
                            
                            tsi_avg_rmse += tsi_rmse
                            tsi_avg_picp += tsi_picp
                            tsi_avg_mpiw += tsi_mpiw
                    
                    if len(ssn_mean_forecast_inv) > 0:
                        ssn_avg_rmse /= len(ssn_mean_forecast_inv)
                        ssn_avg_picp /= len(ssn_mean_forecast_inv)
                        ssn_avg_mpiw /= len(ssn_mean_forecast_inv)
                        
                        tsi_avg_rmse /= len(tsi_mean_forecast_inv)
                        tsi_avg_picp /= len(tsi_mean_forecast_inv)
                        tsi_avg_mpiw /= len(tsi_mean_forecast_inv)
                    
                    print(f"SSN RMSE: {ssn_avg_rmse:.4f}, PICP: {ssn_avg_picp:.4f}, MPIW: {ssn_avg_mpiw:.4f}")
                    print(f"TSI RMSE: {tsi_avg_rmse:.4f}, PICP: {tsi_avg_picp:.4f}, MPIW: {tsi_avg_mpiw:.4f}")
                    
                    last_date = df.index[-1]
                    future_months = (pd.to_datetime('2030-12-01') - last_date).days // 30
                    future_dates = generate_future_dates(last_date, future_months)

                    future_sequence = pca_scaled[-seq_length:]

                    future_forecast_steps = []
                    future_lower_bounds = []
                    future_upper_bounds = []
                    current_sequence = future_sequence.copy()

                    for i in range(0, future_months, forecast_horizon):
                        steps = min(forecast_horizon, future_months - i)
                        
                        all_forecasts = []
                        for model in models:
                            forecast = model.predict(np.array([current_sequence]), verbose=0)
                            forecast = forecast.reshape(-1, n_components)[:steps]
                            all_forecasts.append(forecast)
                        
                        all_forecasts = np.array(all_forecasts)
                        mean_forecast = np.mean(all_forecasts, axis=0)
                        lower_bound = np.quantile(all_forecasts, 0.05, axis=0)
                        upper_bound = np.quantile(all_forecasts, 0.95, axis=0)
                        
                        future_forecast_steps.append(mean_forecast)
                        future_lower_bounds.append(lower_bound)
                        future_upper_bounds.append(upper_bound)
                        
                        current_sequence = np.vstack((current_sequence[steps:], mean_forecast))

                    future_forecasts = np.vstack(future_forecast_steps)
                    future_lowers = np.vstack(future_lower_bounds)
                    future_uppers = np.vstack(future_upper_bounds)

                    future_forecasts_scaled = pca_scaler.inverse_transform(future_forecasts)
                    future_lowers_scaled = pca_scaler.inverse_transform(future_lowers)
                    future_uppers_scaled = pca_scaler.inverse_transform(future_uppers)

                    future_forecasts_original = pca.inverse_transform(future_forecasts_scaled)
                    future_lowers_original = pca.inverse_transform(future_lowers_scaled)
                    future_uppers_original = pca.inverse_transform(future_uppers_scaled)

                    future_ssn = future_forecasts_original[:, 0]
                    future_ssn_lower = future_lowers_original[:, 0]
                    future_ssn_upper = future_uppers_original[:, 0]

                    future_tsi = future_forecasts_original[:, 1]
                    future_tsi_lower = future_lowers_original[:, 1]
                    future_tsi_upper = future_uppers_original[:, 1]
                    
                    plot_forecast(
                        df, val_dates, ssn_mean_forecast_inv, ssn_lower_bound_inv, ssn_upper_bound_inv,
                        future_dates, future_ssn, future_ssn_lower, future_ssn_upper,
                        cycle_dates[24], cycle_dates[25], forecast_horizon, 'ssn',
                        ssn_output_dir, 25-cycle_start+1, seq_length, ssn_avg_rmse, ssn_avg_picp, ssn_avg_mpiw
                    )
                    
                    plot_forecast(
                        df, val_dates, tsi_mean_forecast_inv, tsi_lower_bound_inv, tsi_upper_bound_inv,
                        future_dates, future_tsi, future_tsi_lower, future_tsi_upper,
                        cycle_dates[24], cycle_dates[25], forecast_horizon, 'tsi',
                        tsi_output_dir, 25-cycle_start+1, seq_length, tsi_avg_rmse, tsi_avg_picp, tsi_avg_mpiw
                    )
                    
                    result = {
                        'combination': combination_count,
                        'cycle_start': cycle_start,
                        'seq_length': seq_length,
                        'forecast_horizon': forecast_horizon,
                        'ssn_rmse': ssn_avg_rmse,
                        'ssn_picp': ssn_avg_picp,
                        'ssn_mpiw': ssn_avg_mpiw,
                        'tsi_rmse': tsi_avg_rmse,
                        'tsi_picp': tsi_avg_picp,
                        'tsi_mpiw': tsi_avg_mpiw
                    }
                    results.append(result)
                    
                    with open('forecast_results.csv', 'a', newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow([
                            combination_count, cycle_start, seq_length, forecast_horizon,
                            ssn_avg_rmse, ssn_avg_picp, ssn_avg_mpiw,
                            tsi_avg_rmse, tsi_avg_picp, tsi_avg_mpiw
                        ])
                
                except Exception as e:
                    print(f"Error processing combination: {e}")
                    continue
    
    print("\nTop 5 combinations for SSN based on RMSE:")
    ssn_sorted = sorted(results, key=lambda x: x['ssn_rmse'])
    for i, result in enumerate(ssn_sorted[:5]):
        print(f"{i+1}. Cycles: {25-result['cycle_start']+1}, Seq Length: {result['seq_length']} months, Horizon: {result['forecast_horizon']} months")
        print(f"   RMSE: {result['ssn_rmse']:.4f}, PICP: {result['ssn_picp']:.4f}, MPIW: {result['ssn_mpiw']:.4f}")
    
    print("\nTop 5 combinations for TSI based on RMSE:")
    tsi_sorted = sorted(results, key=lambda x: x['tsi_rmse'])
    for i, result in enumerate(tsi_sorted[:5]):
        print(f"{i+1}. Cycles: {25-result['cycle_start']+1}, Seq Length: {result['seq_length']} months, Horizon: {result['forecast_horizon']} months")
        print(f"   RMSE: {result['tsi_rmse']:.4f}, PICP: {result['tsi_picp']:.4f}, MPIW: {result['tsi_mpiw']:.4f}")

if __name__ == "__main__":
    main()

Running combination 1/105: Cycle 19, Seq Length 60, Horizon 1


/opt/anaconda3/envs/ssn/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SSN RMSE: 3.7254, PICP: 0.3721, MPIW: 5.1186
TSI RMSE: 0.0753, PICP: 0.0000, MPIW: 0.0000
Running combination 2/105: Cycle 19, Seq Length 60, Horizon 3


/opt/anaconda3/envs/ssn/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SSN RMSE: 5.5189, PICP: 0.1951, MPIW: 4.7796
TSI RMSE: 0.0992, PICP: 0.0000, MPIW: 0.0000
Running combination 3/105: Cycle 19, Seq Length 60, Horizon 6


...

SSN RMSE: 6.6324, PICP: 0.6047, MPIW: 11.6652
TSI RMSE: 0.1648, PICP: 0.0000, MPIW: 0.0000
Running combination 104/105: Cycle 23, Seq Length 132, Horizon 3


/opt/anaconda3/envs/ssn/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SSN RMSE: 14.2138, PICP: 0.1545, MPIW: 9.5937
TSI RMSE: 0.2212, PICP: 0.0000, MPIW: 0.0000
Running combination 105/105: Cycle 23, Seq Length 132, Horizon 6


/opt/anaconda3/envs/ssn/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SSN RMSE: 7.6959, PICP: 0.7456, MPIW: 19.6677
TSI RMSE: 0.2200, PICP: 0.0000, MPIW: 0.0000

Top 5 combinations for SSN based on RMSE:
1. Cycles: 7, Seq Length: 96 months, Horizon: 1 months
   RMSE: 3.2134, PICP: 0.4651, MPIW: 6.2283
2. Cycles: 7, Seq Length: 108 months, Horizon: 1 months
   RMSE: 3.3964, PICP: 0.4186, MPIW: 5.5701
3. Cycles: 7, Seq Length: 132 months, Horizon: 1 months
   RMSE: 3.4723, PICP: 0.1395, MPIW: 3.2378
4. Cycles: 7, Seq Length: 60 months, Horizon: 1 months
   RMSE: 3.7254, PICP: 0.3721, MPIW: 5.1186
5. Cycles: 6, Seq Length: 120 months, Horizon: 1 months
   RMSE: 3.7426, PICP: 0.2558, MPIW: 3.7024

Top 5 combinations for TSI based on RMSE:
1. Cycles: 7, Seq Length: 96 months, Horizon: 1 months
   RMSE: 0.0738, PICP: 0.0000, MPIW: 0.0000
2. Cycles: 7, Seq Length: 60 months, Horizon: 1 months
   RMSE: 0.0753, PICP: 0.0000, MPIW: 0.0000
3. Cycles: 7, Seq Length: 84 months, Horizon: 1 months
   RMSE: 0.0790, PICP: 0.0000, MPIW: 0.0000
4. Cycles: 6, Seq Length: 72